In [5]:
import torch
from qpth.qp import QPFunction

# 参数定义
n, d = 100, 3  # 数据点数和模型参数维度
L, H = 2, 2   # ReLU 和 ReHU 项数
lambda_reg = 0.1  # Ridge 正则化参数

# 生成示例数据
X = torch.randn(n, d)  # 输入数据，形状 (n, d)
y = torch.randn(n)     # 目标值，形状 (n,)

# 固定 beta（假设已通过内层优化计算）
beta_star = torch.abs(torch.randn(d))  # 确保 beta >= 0

# 初始化超参数（u_l, v_l, s_h, t_h）
U = torch.randn(L, requires_grad=True)  # u_l，形状 (L,)
V = torch.randn(L, requires_grad=True)  # v_l，形状 (L,)
S = torch.randn(H, requires_grad=True)  # s_h，形状 (H,)
T = torch.randn(H, requires_grad=True)  # t_h，形状 (H,)
tau = torch.ones(H, requires_grad=False)  # T_h 固定为 1，形状 (H,)

def solve_inner_qpth(U, V, S, T, tau, X, y, lambda_reg):
    """
    重建内层 QP，用于计算 beta 对超参数的隐式导数
    """
    # QP 参数
    total_vars = d + L * n + H * n + H * n  # [beta, pi, theta, sigma]
    Q = torch.zeros(total_vars)
    Q[:d] = lambda_reg  # beta 的 Ridge 正则化
    Q[d + L * n:d + L * n + H * n] = 1.0  # theta 的二次项
    Q = torch.diag(Q).unsqueeze(0)
    
    p = torch.zeros(total_vars)
    tau_expanded = tau.repeat(n)  # 形状 (H * n,)
    p[d + L * n + H * n:] = tau_expanded  # sigma 的线性项
    p = p.unsqueeze(0)
    
    # 约束 G z <= h
    G_rows = 2 * L * n + 2 * H * n + d  # 每样本 2L + 2H + d 个约束
    G = torch.zeros(G_rows, total_vars)
    h_values = torch.zeros(G_rows)  # 临时存储 h 的值
    
    # pi_li >= u_l (y_i - x_i^T beta) + v_l
    idx = 0
    for i in range(n):
        for l in range(L):
            G[idx, :d] = U[l] * X[i]  # u_l x_i^T beta，保持计算图
            G[idx, d + l * n + i] = -1.0  # -pi_li
            h_values[idx] = U[l] * y[i] + V[l]  # u_l y_i + v_l，保持计算图
            idx += 1
    
    # pi_li >= 0
    for i in range(n):
        for l in range(L):
            G[idx, d + l * n + i] = -1.0  # -pi_li <= 0
            h_values[idx] = 0.0
            idx += 1
    
    # theta_hi + sigma_hi >= s_h (y_i - x_i^T beta) + t_h
    for i in range(n):
        for h in range(H):
            G[idx, :d] = S[h] * X[i]  # s_h x_i^T beta，保持计算图
            G[idx, d + L * n + h * n + i] = -1.0  # -theta_hi
            G[idx, d + L * n + H * n + h * n + i] = -1.0  # -sigma_hi
            h_values[idx] = S[h] * y[i] + T[h]  # s_h y_i + t_h，保持计算图
            idx += 1
    
    # sigma_hi >= 0
    for i in range(n):
        for h in range(H):
            G[idx, d + L * n + H * n + h * n + i] = -1.0  # -sigma_hi <= 0
            h_values[idx] = 0.0
            idx += 1
    
    # beta_j >= 0
    for j in range(d):
        G[idx, j] = -1.0  # -beta_j <= 0
        h_values[idx] = 0.0
        idx += 1
    
    G = G.unsqueeze(0)
    h = h_values.unsqueeze(0)  # 转换为 (1, G_rows) 形状
    
    # 确保 Q 是 SPD
    Q += 1e-4 * torch.eye(total_vars).unsqueeze(0)
    
    # 使用 qpth 求解
    z = QPFunction(verbose=False)(Q, p, G, h, torch.Tensor(), torch.Tensor())
    beta = z[:, :d].squeeze(0)  # 提取 beta
    return beta

def compute_outer_gradients(beta_star, X, y, U, V, S, T, tau, lambda_reg):
    """
    固定 beta_star，计算外层损失对超参数的导数
    返回：U, V, S, T 的梯度和外层损失值
    """
    # 重建内层 QP，确保 beta 对超参数的依赖
    beta_opt = solve_inner_qpth(U, V, S, T, tau, X, y, lambda_reg)
    
    # 外层损失（MSE）
    y_pred = X @ beta_opt
    L_outer = (1/n) * (y - y_pred).pow(2).sum()
    
    # 计算外层损失的梯度（隐式微分）
    L_outer.backward()
    
    # 提取超参数梯度
    U_grad = U.grad.clone() if U.grad is not None else torch.zeros_like(U)
    V_grad = V.grad.clone() if V.grad is not None else torch.zeros_like(V)
    S_grad = S.grad.clone() if S.grad is not None else torch.zeros_like(S)
    T_grad = T.grad.clone() if T.grad is not None else torch.zeros_like(T)
    
    # 清零梯度（添加保护）
    if U.grad is not None:
        U.grad.zero_()
    if V.grad is not None:
        V.grad.zero_()
    if S.grad is not None:
        S.grad.zero_()
    if T.grad is not None:
        T.grad.zero_()
    
    return U_grad, V_grad, S_grad, T_grad, L_outer.item()

# 简单实验：初始化超参数，计算导数，进行一步梯度下降
if __name__ == "__main__":
    # 初始超参数
    print("初始超参数：")
    print(f"U: {U}")
    print(f"V: {V}")
    print(f"S: {S}")
    print(f"T: {T}")
    
    # 计算外层损失对超参数的导数
    U_grad, V_grad, S_grad, T_grad, outer_loss = compute_outer_gradients(
        beta_star, X, y, U, V, S, T, tau, lambda_reg
    )
    
    # 打印导数
    print("\n超参数导数：")
    print(f"U_grad: {U_grad}")
    print(f"V_grad: {V_grad}")
    print(f"S_grad: {S_grad}")
    print(f"T_grad: {T_grad}")
    print(f"外层损失 (MSE): {outer_loss:.4f}")
    
    # 进行一步梯度下降
    learning_rate = 0.01
    with torch.no_grad():
        U_new = U - learning_rate * U_grad
        V_new = V - learning_rate * V_grad
        S_new = S - learning_rate * S_grad
        T_new = T - learning_rate * T_grad
    
    # 更新超参数
    U = U_new.clone().requires_grad_(True)
    V = V_new.clone().requires_grad_(True)
    S = S_new.clone().requires_grad_(True)
    T = T_new.clone().requires_grad_(True)
    
    # 打印优化后的超参数
    print("\n优化后超参数（一步梯度下降）：")
    print(f"U: {U}")
    print(f"V: {V}")
    print(f"S: {S}")
    print(f"T: {T}")
    
    # 验证 beta 是否保持不变（重新计算 QP 解）
    beta_opt = solve_inner_qpth(U, V, S, T, tau, X, y, lambda_reg)
    print(f"\n重新计算的 beta: {beta_opt}")
    print(f"原 beta_star: {beta_star}")
    print(f"约束满足情况 (beta >= 0): {torch.all(beta_opt >= 0)}")

    # 观察预测的结果和真实值
    y_pred = X @ beta_opt
    print(f"\n预测值：{y_pred}")
    print(f"真实值：{y}")
    print(f"各项误差：{(y - y_pred)}")

初始超参数：
U: tensor([-0.7963,  0.1172], requires_grad=True)
V: tensor([ 0.8294, -0.1393], requires_grad=True)
S: tensor([-1.9455, -0.1206], requires_grad=True)
T: tensor([0.0673, 0.8415], requires_grad=True)

超参数导数：
U_grad: tensor([ 4.2148e-07, -9.8387e-07])
V_grad: tensor([-1.5220e-07,  3.0716e-07])
S_grad: tensor([4.3775e-03, 2.8695e-11])
T_grad: tensor([ 2.1157e-02, -3.1344e-11])
外层损失 (MSE): 0.8964

优化后超参数（一步梯度下降）：
U: tensor([-0.7963,  0.1172], requires_grad=True)
V: tensor([ 0.8294, -0.1393], requires_grad=True)
S: tensor([-1.9456, -0.1206], requires_grad=True)
T: tensor([0.0671, 0.8415], requires_grad=True)

重新计算的 beta: tensor([-1.6282e-08,  1.5359e-01, -2.4855e-08], grad_fn=<SqueezeBackward1>)
原 beta_star: tensor([0.9384, 1.3597, 0.0404])
约束满足情况 (beta >= 0): False

预测值：tensor([ 0.0855, -0.0629,  0.0185,  0.2969,  0.0069,  0.0090, -0.2543, -0.1480,
         0.2114, -0.0606,  0.4863,  0.0932,  0.0223, -0.3070, -0.0892, -0.0684,
        -0.1343, -0.2777, -0.0868, -0.0420,  0.2291,  0.0

In [7]:
import torch
from qpth.qp import QPFunction

def build_qp_matrices(U, V, S, T, tau, X, y, lambda_reg):
    r"""
    构建 Q, p, G, h 矩阵：
    - 内层问题变量: z = [beta, pi, theta, sigma]
    - 目标函数:  (1/2) * z^T Q z + p^T z
    - 约束: G z <= h
    
    其中:
      - beta 维度为 d
      - pi 维度为 L * n
      - theta 维度为 H * n
      - sigma 维度为 H * n
    """
    n, d = X.shape
    L = U.shape[0]
    H = S.shape[0]
    
    # total_vars = d + (L*n) + (H*n) + (H*n)
    total_vars = d + L * n + 2 * H * n
    
    #-------------------------------------
    # 1) 构建 Q (二次项)
    #   Q 对角线各部分对应: [lambda_reg * I_d, 0, I_{H*n}, I_{H*n}],
    #   pi_li 不含二次项，theta_hi 含 1.0, sigma_hi 含 1.0
    #-------------------------------------
    Q_diag = torch.zeros(total_vars, dtype=X.dtype, device=X.device)
    # beta的正则化
    Q_diag[:d] = lambda_reg
    # theta 位置
    Q_diag[d + L * n : d + L * n + H * n] = 1.0
    # sigma 位置
    Q_diag[d + L * n + H * n : d + L * n + 2 * H * n] = 1.0
    
    Q = torch.diag(Q_diag).unsqueeze(0)
    
    #-------------------------------------
    # 2) 构建 p (线性项)
    #   只在 sigma 上有 tau * sigma
    #-------------------------------------
    p = torch.zeros(total_vars, dtype=X.dtype, device=X.device)
    p[d + L * n + H * n : ] = tau.repeat(n)
    p = p.unsqueeze(0)
    
    #-------------------------------------
    # 3) 构建 G, h (不等式约束)
    #   约束总数 = n*(2L + 2H) + d
    #    -- pi_li >= U_l(y_i - x_i^T beta) + V_l
    #    -- pi_li >= 0
    #    -- theta_hi + sigma_hi >= S_h(y_i - x_i^T beta) + T_h
    #    -- sigma_hi >= 0
    #    -- beta_j >= 0
    #-------------------------------------
    G_rows = 2 * L * n + 2 * H * n + d
    G = torch.zeros(G_rows, total_vars, dtype=X.dtype, device=X.device)
    h_values = torch.zeros(G_rows, dtype=X.dtype, device=X.device)
    
    row_idx = 0
    
    # (a) pi_li >= U_l (y_i - x_i^T beta) + V_l
    #     => -pi_li + U_l x_i^T beta <= U_l y_i + V_l
    # 故 G[row_idx, :d] = U[l] * x_i
    #    G[row_idx, d + l*n + i] = -1
    #    h_values[row_idx] = U[l] * y[i] + V[l]
    for i in range(n):
        for l in range(L):
            G[row_idx, :d] = U[l] * X[i]
            G[row_idx, d + l*n + i] = -1.0
            h_values[row_idx] = U[l] * y[i] + V[l]
            row_idx += 1
    
    # (b) pi_li >= 0 => -pi_li <= 0
    for i in range(n):
        for l in range(L):
            G[row_idx, d + l*n + i] = -1.0
            h_values[row_idx] = 0.0
            row_idx += 1
    
    # (c) theta_hi + sigma_hi >= S_h (y_i - x_i^T beta) + T_h
    #     => -theta_hi - sigma_hi + S_h x_i^T beta <= S_h y_i + T_h
    for i in range(n):
        for h_ in range(H):
            G[row_idx, :d] = S[h_] * X[i]
            G[row_idx, d + L*n + h_*n + i] = -1.0   # theta
            G[row_idx, d + L*n + H*n + h_*n + i] = -1.0  # sigma
            h_values[row_idx] = S[h_] * y[i] + T[h_]
            row_idx += 1
    
    # (d) sigma_hi >= 0 => -sigma_hi <= 0
    for i in range(n):
        for h_ in range(H):
            idx_sigma = d + L*n + H*n + h_*n + i
            G[row_idx, idx_sigma] = -1.0
            h_values[row_idx] = 0.0
            row_idx += 1
    
    # (e) beta_j >= 0 => -beta_j <= 0
    for j in range(d):
        G[row_idx, j] = -1.0
        h_values[row_idx] = 0.0
        row_idx += 1
    
    G = G.unsqueeze(0)
    h = h_values.unsqueeze(0)
    
    #-------------------------------------
    # 为 Q 增加扰动，避免数值不稳定
    #-------------------------------------
    eps = 1e-4
    Q = Q + eps * torch.eye(total_vars, dtype=X.dtype, device=X.device).unsqueeze(0)
    
    return Q, p, G, h

def solve_inner_qpth(U, V, S, T, tau, X, y, lambda_reg):
    r"""
    给定超参数 (U, V, S, T, tau) 和数据 (X, y)，
    构建并求解内层 QP。
    返回: 内层最优解 beta (大小为 d)。
    """
    # 构建 Q, p, G, h
    Q, p, G, h = build_qp_matrices(U, V, S, T, tau, X, y, lambda_reg)
    
    # 调用 QPFunction 做求解
    z = QPFunction(verbose=False)(Q, p, G, h, torch.empty(0), torch.empty(0))
    
    # 提取前 d 维为 beta
    n, d = X.shape
    beta_opt = z[:, :d].squeeze(0)
    return beta_opt

def compute_outer_gradients(
    X, y,
    U, V, S, T, tau,
    lambda_reg
):
    r"""
    计算外层目标 (此处为 MSE) 对超参数的梯度。
    做法：
    1) 基于给定超参数解内层 QP，得到 beta_opt
    2) 用 beta_opt 计算外层损失 MSE
    3) backward() 得到对 U, V, S, T 的梯度
    """
    beta_opt = solve_inner_qpth(U, V, S, T, tau, X, y, lambda_reg)
    n = X.shape[0]
    
    # 外层损失: MSE
    y_pred = X @ beta_opt
    loss_outer = (1.0 / n) * (y - y_pred).pow(2).sum()
    
    # 反向传播
    loss_outer.backward()
    
    # 收集梯度
    U_grad = U.grad.detach().clone() if U.grad is not None else torch.zeros_like(U)
    V_grad = V.grad.detach().clone() if V.grad is not None else torch.zeros_like(V)
    S_grad = S.grad.detach().clone() if S.grad is not None else torch.zeros_like(S)
    T_grad = T.grad.detach().clone() if T.grad is not None else torch.zeros_like(T)
    
    # 梯度清零
    if U.grad is not None:
        U.grad.zero_()
    if V.grad is not None:
        V.grad.zero_()
    if S.grad is not None:
        S.grad.zero_()
    if T.grad is not None:
        T.grad.zero_()
    
    return {
        "U_grad": U_grad,
        "V_grad": V_grad,
        "S_grad": S_grad,
        "T_grad": T_grad,
        "loss": loss_outer.item(),
        "beta_opt": beta_opt.detach().clone()
    }

def main():
    # 数据与超参数初始化
    n, d = 100, 3
    L, H = 2, 2
    lambda_reg = 0.1
    
    X = torch.randn(n, d)
    y = torch.randn(n)
    
    # 假设内层 beta_star 已知: (此处不太用到, 但留着也可)
    beta_star = torch.abs(torch.randn(d))
    
    U = torch.randn(L, requires_grad=True)
    V = torch.randn(L, requires_grad=True)
    S = torch.randn(H, requires_grad=True)
    T = torch.randn(H, requires_grad=True)
    tau = torch.ones(H, requires_grad=False)  # 通常做固定
    
    print("初始超参数:")
    print(f"U: {U}")
    print(f"V: {V}")
    print(f"S: {S}")
    print(f"T: {T}")
    
    # 计算外层损失对超参数的导数
    results = compute_outer_gradients(X, y, U, V, S, T, tau, lambda_reg)
    U_grad = results["U_grad"]
    V_grad = results["V_grad"]
    S_grad = results["S_grad"]
    T_grad = results["T_grad"]
    outer_loss = results["loss"]
    
    print("\n超参数导数:")
    print(f"U_grad: {U_grad}")
    print(f"V_grad: {V_grad}")
    print(f"S_grad: {S_grad}")
    print(f"T_grad: {T_grad}")
    print(f"外层损失(MSE): {outer_loss:.4f}")
    
    # 做一步梯度下降
    lr = 0.01
    with torch.no_grad():
        U -= lr * U_grad
        V -= lr * V_grad
        S -= lr * S_grad
        T -= lr * T_grad
    
    # 更新需要 requires_grad
    U.requires_grad_(True)
    V.requires_grad_(True)
    S.requires_grad_(True)
    T.requires_grad_(True)
    
    print("\n优化后超参数（一步梯度下降）:")
    print(f"U: {U}")
    print(f"V: {V}")
    print(f"S: {S}")
    print(f"T: {T}")
    
    # 重新验证内层解
    beta_opt = solve_inner_qpth(U, V, S, T, tau, X, y, lambda_reg)
    print(f"\n重新计算的 beta: {beta_opt}")
    print(f"原 beta_star: {beta_star}")
    print(f"约束满足情况 (beta >= 0): {torch.all(beta_opt >= 0)}")
    
    # 观察预测结果
    y_pred = X @ beta_opt
    print(f"\n预测值: {y_pred}")
    print(f"真实值: {y}")
    print(f"各项误差: {y - y_pred}")

if __name__ == "__main__":
    main()


初始超参数:
U: tensor([-0.9723, -0.4389], requires_grad=True)
V: tensor([ 0.6166, -0.1783], requires_grad=True)
S: tensor([ 0.9274, -0.7048], requires_grad=True)
T: tensor([1.9331, 1.2874], requires_grad=True)

超参数导数:
U_grad: tensor([2.3499e-06, 1.4791e-06])
V_grad: tensor([ 1.0751e-06, -3.8228e-08])
S_grad: tensor([0.0103, 0.0023])
T_grad: tensor([-0.0042,  0.0002])
外层损失(MSE): 1.1211

优化后超参数（一步梯度下降）:
U: tensor([-0.9723, -0.4389], requires_grad=True)
V: tensor([ 0.6166, -0.1783], requires_grad=True)
S: tensor([ 0.9273, -0.7048], requires_grad=True)
T: tensor([1.9331, 1.2874], requires_grad=True)

重新计算的 beta: tensor([3.2026e-07, 5.9602e-07, 3.7011e-02], grad_fn=<SqueezeBackward1>)
原 beta_star: tensor([1.3249, 1.0254, 1.0872])
约束满足情况 (beta >= 0): True

预测值: tensor([-0.0157,  0.0306, -0.0022, -0.0161, -0.0750,  0.0298,  0.0312,  0.0044,
         0.0346, -0.0289, -0.0788,  0.0546,  0.0109,  0.0192,  0.0818, -0.0532,
        -0.0780,  0.0191, -0.0247,  0.0530,  0.0353,  0.0644, -0.0176, -0.0667,